In [37]:
import pandas as pd
from scholarly import scholarly
from scholarly import ProxyGenerator
import os
import csv

In [ ]:
# setup free proxy, lowkey doesn't work very well so I suggest using ScraperAPI
# you can just keep getting free trials froms ScraperAPI lmao
pg = ProxyGenerator()
pg.FreeProxies()
scholarly.use_proxy(pg)

In [38]:
# how to set up ScraperAPI proxy, use the key that it gives u when u sign up
pg = ProxyGenerator()
success = pg.ScraperAPI('2c8ddef96ccd204be4e734417dc98f03')
scholarly.use_proxy(pg)

In [39]:
# list of professors to scrape, will probably add their scholar profile id bc its easier to find them
QTM_professors1 = [
    'zvOPm8gAAAAJ', # Weihua An
    'U_xI7eAAAAAJ', # Michal Arbilly
    '9zsMl5kAAAAJ', # Hun Chung
    'LB9hecMAAAAJ', # Jacopo Di Iorio
    '9s1slLcAAAAJ', # Danilo Freire
    '30u0V1EAAAAJ', # Adam Glynn
    'JnRNZccAAAAJ', # Jo Guldi
    'lTDtAqoAAAAJ', # Craig Hadley
    '_QUUb-gAAAAJ', # David Hirshberg
    'Xc9Sco0AAAAJ', # Lauren Klein
    'Q5KlCMwAAAAJ' # Kevin McAlister
]

QTM_professors2 = [
    'dk_JiwgAAAAJ', # Pablo Montagnes
    'rhSQ6egAAAAJ', # Gregory Palermo
    'zEq533oAAAAJ', # John Patty
    'XD8LHcgAAAAJ', # Kevin Quinn
    'yFPu-mkAAAAJ', # Alejandro Sanchez Becerra
    '_OzUlMkAAAAJ', # Sandeep Soni
    'J4NQQIEAAAAJ', # Allison Stashko
    'vgfVoq4AAAAJ', # Joseph Sutherland
    'OGjoFXcAAAAJ', # Alexander Tolbert
    'lg_0u-0AAAAJ' # Ruoxuan Xiong
]


In [ ]:
CS_professors1 = [
    '3BX3vWcAAAAJ', # Eugene Agichtein
    '2jEFz5EAAAAJ', # Dorian Arnold
    'aL6XtIUAAAAJ', # Yana Bromberg
    'xdddblAAAAAJ', # Jinho Choi
    'AIf9mB4AAAAJ', # Nosayba El-Sayed
    'EQ61oMAAAAAJ', # Davide Fossati
    'BOFfWR0AAAAJ', # Zhichun Guo
    'DrUBb5sAAAAJ', # Joyce Ho
    'eWow24EAAAAJ', # Wei Jin
    'cAE-vgwAAAAJ', # Joonseok Kim
    '22ohn6AAAAAJ', # Fei Liu
    'RxcQjoYAAAAJ', # Yolanda Rankin
    '-6bAV2cAAAAJ', # Kai Shu
    'a_z5a5wAAAAJ', # Shengpu Tang
    'YfcphqMAAAAJ', # Emily Wall
    'yGtJEDsAAAAJ', # Kristin Williams
    'jJ8BLgsAAAAJ', # Li Xiong
    'mOINlwcAAAAJ', # Carl Yang
    'qnvyqtwAAAAJ', # Liang Zhao
    's-FDyf4AAAAJ' # Andreas Zufle
]

In [ ]:
# getting the author profile from id (Weihua An)
author = scholarly.search_author_id('zvOPm8gAAAAJ')

In [23]:
# Fill the author object with the authors publications
author = scholarly.fill(author, sections=['publications'])
scholarly.pprint(author)

b'{\'affiliation\': \'Emory University, Sociology & Quantitative Theory and Methods\',\n \'citedby\': 1304,\n \'email_domain\': \'@emory.edu\',\n \'filled\': False,\n \'homepage\': \'http://weihuaan.net/\',\n \'interests\': [\'Social Networks\',\n               \'Causal Inference\',\n               \'Social Policy\',\n               \'Health\',\n               \'Organizations\'],\n \'name\': \'Weihua An\',\n \'organization\': 7693822459085397235,\n \'publications\': [{\'author_pub_id\': \'zvOPm8gAAAAJ:d1gkVwhDpl0C\',\n                   \'bib\': {\'citation\': \'Social forces 92 (1), 303-327, 2013\',\n                           \'pub_year\': \'2013\',\n                           \'title\': \'Evicting Children\'},\n                   \'citedby_url\': \'https://scholar.google.com/scholar?oi=bibs&hl=en&cites=2730129611184238020\',\n                   \'cites_id\': [\'2730129611184238020\'],\n                   \'filled\': False,\n                   \'num_citations\': 203,\n               

In [25]:
# filter out the publications that are older than 2020 before filling
filtered_publications = [
    pub for pub in author.get('publications', [])
    if int(pub.get('bib', {}).get('pub_year', 0)) >= 2020]

print(filtered_publications)

[{'container_type': 'Publication', 'source': <PublicationSource.AUTHOR_PUBLICATION_ENTRY: 'AUTHOR_PUBLICATION_ENTRY'>, 'bib': {'title': 'Causal network analysis', 'pub_year': '2022', 'citation': 'Annual Review of Sociology 48 (1), 23-41, 2022'}, 'filled': False, 'author_pub_id': 'zvOPm8gAAAAJ:-jrNzM816MMC', 'num_citations': 35, 'citedby_url': 'https://scholar.google.com/scholar?oi=bibs&hl=en&cites=13437606084044806831', 'cites_id': ['13437606084044806831']}, {'container_type': 'Publication', 'source': <PublicationSource.AUTHOR_PUBLICATION_ENTRY: 'AUTHOR_PUBLICATION_ENTRY'>, 'bib': {'title': 'Friendship network formation in Chinese middle schools: Patterns of inequality and homophily', 'pub_year': '2022', 'citation': 'Social Networks 68, 218-228, 2022'}, 'filled': False, 'author_pub_id': 'zvOPm8gAAAAJ:MAUkC_7iAq8C', 'num_citations': 20, 'citedby_url': 'https://scholar.google.com/scholar?oi=bibs&hl=en&cites=13873405708389654494', 'cites_id': ['13873405708389654494']}, {'container_type': 

In [26]:
publication_data = []
# loops through each filtered publication and fills it with the required information 
# (takes a while depending on how many papers)
for pub in filtered_publications:
    try:
        filled_pub = scholarly.fill(pub)
        bib = filled_pub.get('bib', {})
        title = bib.get('title', '')
        year = bib.get('pub_year', '')
        abstract = bib.get('abstract', '')
        url = filled_pub.get('pub_url', '')

        publication_data.append({
            'Author': author.get('name', ''),
            'Title': title,
            'Year': year,
            'URL': url,
            'Abstract': abstract
        })
    except Exception as e:
        print(f"Error filling publication: {e}")

publications_df = pd.DataFrame(publication_data)

# df that has the publications of the author
display(publications_df)

,Author,Title,Year,URL,Abstract
0,Weihua An,Causal network analysis,2022,https://www.annualreviews.org/content/journals...,Fueled by recent advances in statistical model...
1,Weihua An,Friendship network formation in Chinese middle...,2022,https://www.sciencedirect.com/science/article/...,Previous studies on adolescent friendship netw...
2,Weihua An,"You said, they said: A framework on informant ...",2022,https://www.sciencedirect.com/science/article/...,Informants have been used extensively to provi...
3,Weihua An,Opening the blackbox of treatment interference...,2022,https://journals.sagepub.com/doi/abs/10.1177/0...,Causal inference under treatment interference ...
4,Weihua An,Methodological advances in quantitative social...,2023,https://pubmed.ncbi.nlm.nih.gov/36797000/,Methodological advances in quantitative social...
5,Weihua An,"Fear not scarcity but inequality, not poverty ...",2021,https://journals.sagepub.com/doi/abs/10.1177/0...,"This special issue, “New Quantitative Approach..."
6,Weihua An,A tale of twin dependence: a new multivariate ...,2023,https://journals.sagepub.com/doi/abs/10.1177/0...,"In this article, I present a new multivariate ..."
7,Weihua An,Characterization of trajectories of physical a...,2023,https://link.springer.com/article/10.1186/s128...,Cigarette smoking and physical inactivity are ...
8,Weihua An,"Race, state surveillance, and policy spillover...",2023,https://academic.oup.com/sf/article-abstract/1...,This paper investigates whether restrictive im...
9,Weihua An,"The nearness between us, and the space within ...",2020,https://journals.sagepub.com/doi/abs/10.1177/0...,The year 2020 has been unprecedented. As a res...


In [33]:
def scrape_recent_publications(professor_ids):
    all_publications = []
    for prof_id in professor_ids:
        try:
            print(f"Fetching author: {prof_id}")
            author = scholarly.search_author_id(prof_id)
            author = scholarly.fill(author, sections=['publications'])
            name = author.get('name', 'Unknown')

            filtered_pubs = [
                pub for pub in author.get('publications', [])
                if int(pub.get('bib', {}).get('pub_year', 0)) >= 2020
            ]

            for pub in filtered_pubs:
                try:
                    filled_pub = scholarly.fill(pub)
                    bib = filled_pub.get('bib', {})
                    all_publications.append({
                        'Author': name,
                        'Title': bib.get('title', ''),
                        'Year': bib.get('pub_year', ''),
                        'URL': filled_pub.get('pub_url', ''),
                        'Abstract': bib.get('abstract', '')
                    })
                except Exception as e:
                    print(f"Error filling publication for {name}: {e}")
        except Exception as e:
            print(f"Error fetching author with ID {prof_id}: {e}")

    return pd.DataFrame(all_publications)

In [43]:
def scrape_professors_to_csv(professor_ids, start_year=2020, csv_file='publications.csv', log_file='processed_ids.txt'):
    # Create CSV file with headers if it doesn't exist
    if not os.path.exists(csv_file):
        with open(csv_file, mode='w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['Author', 'Title', 'Year', 'URL', 'Abstract'])
            writer.writeheader()

    # Load processed IDs to avoid re-processing
    processed_ids = set()
    if os.path.exists(log_file):
        with open(log_file, 'r') as f:
            processed_ids = set(line.strip() for line in f.readlines())

    for prof_id in professor_ids:
        if prof_id in processed_ids:
            print(f"Skipping already processed ID: {prof_id}")
            continue

        try:
            print(f"Fetching author: {prof_id}")
            author = scholarly.search_author_id(prof_id)
            author = scholarly.fill(author, sections=['publications'])
            publications = author.get('publications', [])

            filtered_publications = [
                pub for pub in publications
                if int(pub.get('bib', {}).get('pub_year', 0)) >= start_year
            ]

            with open(csv_file, mode='a', newline='', encoding='utf-8') as f:
                writer = csv.DictWriter(f, fieldnames=['Author', 'Title', 'Year', 'URL', 'Abstract'])

                for pub in filtered_publications:
                    try:
                        filled_pub = scholarly.fill(pub)
                        bib = filled_pub.get('bib', {})
                        writer.writerow({
                            'Author': author.get('name', ''),
                            'Title': bib.get('title', ''),
                            'Year': bib.get('pub_year', ''),
                            'URL': filled_pub.get('pub_url', ''),
                            'Abstract': bib.get('abstract', '')
                        })
                    except Exception as e:
                        print(f"[{author.get('name', 'Unknown')}]: Error filling publication: {e}")

            # Append processed ID to the log
            with open(log_file, 'a') as log_f:
                log_f.write(f"{prof_id}\n")

        except Exception as e:
            print(f"Error processing author ID {prof_id}: {e}")

In [44]:
scrape_professors_to_csv(QTM_professors1, csv_file='QTM_publications.csv')

Skipping already processed ID: zvOPm8gAAAAJ
Fetching author: U_xI7eAAAAAJ
Fetching author: 9zsMl5kAAAAJ
Fetching author: LB9hecMAAAAJ
Fetching author: 9s1slLcAAAAJ
Fetching author: 30u0V1EAAAAJ
Fetching author: JnRNZccAAAAJ
Fetching author: lTDtAqoAAAAJ


KeyboardInterrupt: 

In [42]:
publications_df = pd.read_csv('QTM_publications.csv')
display(publications_df)

,Author,Title,Year,URL,Abstract
0,Weihua An,Causal network analysis,2022,https://www.annualreviews.org/content/journals...,Fueled by recent advances in statistical model...
1,Weihua An,Friendship network formation in Chinese middle...,2022,https://www.sciencedirect.com/science/article/...,Previous studies on adolescent friendship netw...
2,Weihua An,"You said, they said: A framework on informant ...",2022,https://www.sciencedirect.com/science/article/...,Informants have been used extensively to provi...
3,Weihua An,Opening the blackbox of treatment interference...,2022,https://journals.sagepub.com/doi/abs/10.1177/0...,Causal inference under treatment interference ...
4,Weihua An,Methodological advances in quantitative social...,2023,https://pubmed.ncbi.nlm.nih.gov/36797000/,Methodological advances in quantitative social...
5,Weihua An,"Fear not scarcity but inequality, not poverty ...",2021,https://journals.sagepub.com/doi/abs/10.1177/0...,"This special issue, “New Quantitative Approach..."
6,Weihua An,A tale of twin dependence: a new multivariate ...,2023,https://journals.sagepub.com/doi/abs/10.1177/0...,"In this article, I present a new multivariate ..."
7,Weihua An,Characterization of trajectories of physical a...,2023,https://link.springer.com/article/10.1186/s128...,Cigarette smoking and physical inactivity are ...
8,Weihua An,"Race, state surveillance, and policy spillover...",2023,https://academic.oup.com/sf/article-abstract/1...,This paper investigates whether restrictive im...
9,Weihua An,"The nearness between us, and the space within ...",2020,https://journals.sagepub.com/doi/abs/10.1177/0...,The year 2020 has been unprecedented. As a res...
